# Data Loading

In [965]:
#package installation
! pip install pandas numpy matplotlib seaborn openpyxl
! pip install matplotlib seaborn


In [966]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
from sklearn.preprocessing import StandardScaler

In [967]:
#load data from MongoDB?

# CSV Files
reviews_df = pd.read_csv("../../data/all_countries_reviews.csv")
trends_df = pd.read_csv("../../data/google_trends_data.csv")
currency_df = pd.read_csv("../../data/southeast_asia_currency_monthly_avg.csv")

# Excel File
visitors_df = pd.read_excel("../../data/monthly_num_visitors.xlsx")

# JSON Files
with open("../../data/reddit_comments_20250326_161503.json", "r") as f:
    reddit_comments = json.load(f)

with open("../../data/reddit_submissions_20250326_161503.json", "r") as f:
    reddit_submissions = json.load(f)

with open("../../data/country_posts.json", "r") as f:
    country_posts = json.load(f)

with open("../../data/sentiment_analysis(reddit).json", "r") as f:
    reddit_sentiment = json.load(f)

with open("../../data/sentiment_analysis(instagram).json", "r") as f:
    ig_sentiment = json.load(f)



In [968]:
#Preview all CSV data
#for name, df in [("Reviews", reviews_df), ("Google Trends", trends_df), ("Currency", currency_df), ("Visitors", visitors_df)]:
#   print(f"\n--- {name} ---")
#   display(df.head())
#   display(df.info())


In [969]:
# Explore sentiment data
print("\n--- Reddit Sentiment Analysis ---")
print(reddit_sentiment[0])

print("\n--- Instagram Sentiment Analysis ---")
print(ig_sentiment[0])



--- Reddit Sentiment Analysis ---
{'submission_id': '1jk3mug', 'author': 'PennSt8_08', 'created_utc': '2025-03-26T04:26:36.000Z', 'month_year': '03-2025', 'name': 't3_1jk3mug', 'num_comments': 6, 'score': 0, 'selftext': 'I’m a US citizen who just purchased $1000 worth of clothes in Vietnam. I’ll soon be traveling to Cambodia then Singapore for a few days before returning home. I understand in the US I’ll have to make a declaration as I’m over the $800 threshold. What about the stops I have between now and then? I assume I need to declare at each subsequent country but they won’t charge me for them as I’m not staying or selling these goods there. To avoid any confusion or risk of that, I’m considering paying to just ship items home from Vietnam but I suspect if I carry them back with me to the US and verbally declare they may not bother charging me since I’m barely over the limit. Thanks in advance if anyone can help shed some light or suggest the best way to handle this.', 'subreddit_

# Prepare individual dataframes


In [970]:
reddit_df = pd.DataFrame(reddit_sentiment)
reddit_df.head(5)

,submission_id,author,created_utc,month_year,name,num_comments,score,selftext,subreddit_name,title,...,mentioned_cities,date_only,time_only,day_of_week,month,year,is_weekend,weekday_name,cleaned_caption,sentiment_score
0,1jk3mug,PennSt8_08,2025-03-26T04:26:36.000Z,03-2025,t3_1jk3mug,6,0,I’m a US citizen who just purchased $1000 wort...,travel,Do I have to pay taxes / duty at each country ...,...,singapore,2025-03-26T00:00:00.000,04:26:36,2,3,2025,0,Wednesday,Im a US citizen who just purchased 1000 worth ...,0.6192
1,1jjxshr,rematto,2025-03-25T23:31:26.000Z,03-2025,t3_1jjxshr,10,0,Hi All!\n\nI was wondering if you could share ...,travel,Spain Travel in August: Unique Activities?,...,,2025-03-25T00:00:00.000,23:31:26,1,3,2025,0,Tuesday,Hi All! I was wondering if you could share som...,0.9931
2,1jjuxn5,After-Belt5424,2025-03-25T21:29:53.000Z,03-2025,t3_1jjuxn5,14,1,Hi all! I'm a female in my early 20s with 4 da...,travel,Bangkok/Pattaya vs Singapore for 4 day side-tr...,...,"bangkok, singapore",2025-03-25T00:00:00.000,21:29:53,1,3,2025,0,Tuesday,Hi all! Im a female in my early 20s with 4 day...,0.8478
3,1jjuq8t,Stock_Rabbit_1901,2025-03-25T21:21:30.000Z,03-2025,t3_1jjuq8t,5,6,I’m going to China in May to visit a manufactu...,travel,"China trip, advice for 6 nights",...,"beijing, hangzhou, shanghai",2025-03-25T00:00:00.000,21:21:30,1,3,2025,0,Tuesday,Im going to China in May to visit a manufactur...,0.9627
4,1jjub8j,leopardspot20,2025-03-25T21:04:22.000Z,03-2025,t3_1jjub8j,11,5,We (retired couple) are thinking of going to J...,travel,Japan in winter 6 weeks- where to stay?,...,"kyoto, tokyo",2025-03-25T00:00:00.000,21:04:22,1,3,2025,0,Tuesday,We retired couple are thinking of going to Jap...,0.9267


In [971]:
columns_to_keep = [
    "month_year",
    "mentioned_countries",
    "mentioned_cities",
    "sentiment_score",
]
reddit_df = reddit_df[columns_to_keep]
reddit_df.head(5)

,month_year,mentioned_countries,mentioned_cities,sentiment_score
0,03-2025,"singapore, vietnam, cambodia",singapore,0.6192
1,03-2025,thailand,,0.9931
2,03-2025,"singapore, india, thailand","bangkok, singapore",0.8478
3,03-2025,china,"beijing, hangzhou, shanghai",0.9627
4,03-2025,japan,"kyoto, tokyo",0.9267


In [972]:
reddit_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 921 entries, 0 to 920
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   month_year           921 non-null    object 
 1   mentioned_countries  921 non-null    object 
 2   mentioned_cities     921 non-null    object 
 3   sentiment_score      921 non-null    float64
dtypes: float64(1), object(3)
memory usage: 28.9+ KB


In [973]:
#changed to the same month_year format
reddit_df["month_year"] = pd.to_datetime(reddit_df["month_year"], errors="coerce")
reddit_df["month_year"] = reddit_df["month_year"].dt.strftime("%Y-%m")

/var/folders/85/3q25ml057m51x2d6fx8s4w740000gn/T/ipykernel_35767/3486893326.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  reddit_df["month_year"] = pd.to_datetime(reddit_df["month_year"], errors="coerce")


In [974]:
#parse the str countries into list
reddit_df["mentioned_countries"] = reddit_df["mentioned_countries"].apply(
    lambda x: [c.strip().lower() for c in x.split(",")] if isinstance(x, str) else x
)
#explode the list so as to aggregate based on countries
reddit_df_exploded = reddit_df.explode("mentioned_countries")
print("Before explode:", reddit_df.shape[0])
print("After explode:", reddit_df_exploded.shape[0])


Before explode: 921
After explode: 2185


In [975]:
reddit_df_exploded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2185 entries, 0 to 920
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   month_year           2185 non-null   object 
 1   mentioned_countries  2185 non-null   object 
 2   mentioned_cities     2185 non-null   object 
 3   sentiment_score      2185 non-null   float64
dtypes: float64(1), object(3)
memory usage: 85.4+ KB


In [976]:
#aggregate average country sentiment score
avg_sentiment = reddit_df_exploded.groupby(
    ["mentioned_countries", "month_year"]
)["sentiment_score"].mean().reset_index()
avg_sentiment.columns = ["country", "month_year", "reddit_sentiment"]


In [977]:
#capitalise the first letter so as to merge with other datasets
avg_sentiment["country"] = avg_sentiment["country"].str.title()
avg_sentiment.head(5)



,country,month_year,reddit_sentiment
0,Brunei,2025-02,0.765371
1,Cambodia,2023-10,0.000000
2,Cambodia,2024-03,0.000000
3,Cambodia,2025-01,0.815660
4,Cambodia,2025-02,0.778042


In [978]:
##continue to parse the str cities into list if using cities
##reddit_df_exploded["mentioned_cities"] = reddit_df_exploded["mentioned_cities"].apply(
##    lambda x: [c.strip().lower() for c in x.split(",")] if isinstance(x, str) else x
##)
##reddit_df_exploded_cc = reddit_df_exploded.explode("mentioned_cities")
##print("Before explode:", reddit_df_exploded.shape[0])
##print("After explode:", reddit_df_exploded_cc.shape[0])

In [979]:
ig_df = pd.DataFrame(ig_sentiment)
ig_df.head(5)

,caption,comment_count,date,like_count,play_count,country,location,date_only,time_only,day_of_week,month,year,is_weekend,weekday_name,cleaned_caption,language,sentiment_score
0,Just wrapped up an epic core & full-body sessi...,22,2025-01-07T17:58:32.000Z,1259,0,bali,None,2025-01-07T00:00:00.000,17:58:32,1,1,2025,0,Tuesday,Just Wrapped Up An Epic Core Fullbody Session ...,en,0.8302
1,Случайные кадры с Бали 🌿 Какой кадр нравится б...,30,2022-08-04T03:04:51.000Z,559,0,bali,None,2022-08-04T00:00:00.000,03:04:51,3,8,2022,0,Thursday,Random shots with Bali What frame do you like ...,en,0.3612
2,ここ数年でバリ島ではトリュフチョコを扱うチョコレート専門店が増えましたが、こちらもその一つ！...,2,2023-04-29T20:40:00.000Z,559,0,bali,None,2023-04-29T00:00:00.000,20:40:00,5,4,2023,1,Saturday,"Over the past few years, there have been an in...",en,0.9847
3,Nggak heran lagi deh sama Hyoyeon kalo tiba-ti...,26,2023-10-18T01:46:08.000Z,1985,120120,bali,None,2023-10-18T00:00:00.000,01:46:08,2,10,2023,0,Wednesday,No wonder again with Hyoyeon if suddenly in Ba...,en,0.5189
4,"📍Cretya Ubud, Bali 🇮🇩\n.\n.\n.\n#cretyaubud #c...",65,2024-12-07T11:59:29.000Z,936,0,bali,None,2024-12-07T00:00:00.000,11:59:29,5,12,2024,1,Saturday,"Cretya Ubud, Bali . . . cretyaubud cretya cret...",en,0.0000


In [980]:
ig_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1432 entries, 0 to 1431
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   caption          1432 non-null   object 
 1   comment_count    1432 non-null   int64  
 2   date             1432 non-null   object 
 3   like_count       1432 non-null   int64  
 4   play_count       1432 non-null   int64  
 5   country          1432 non-null   object 
 6   location         106 non-null    object 
 7   date_only        1432 non-null   object 
 8   time_only        1432 non-null   object 
 9   day_of_week      1432 non-null   int64  
 10  month            1432 non-null   int64  
 11  year             1432 non-null   int64  
 12  is_weekend       1432 non-null   int64  
 13  weekday_name     1432 non-null   object 
 14  cleaned_caption  1432 non-null   object 
 15  language         1432 non-null   object 
 16  sentiment_score  1432 non-null   float64
dtypes: float64(1),

In [981]:
ig_df['country'].unique() # the countries are cities in fact

array(['bali', 'bangkok', 'beijing', 'hanoi', 'hong_kong', 'jakarta',
       'kuala_lumpur', 'kyoto', 'manila', 'singapore', 'tokyo'],
      dtype=object)

In [982]:
# --- Prepare ig_sentiment ---
##map cities to countries
city_to_country = {
    'bali': 'Indonesia',
    'bangkok': 'Thailand',
    'beijing': 'China',
    'hanoi': 'Vietnam',
    'hong_kong': 'Hong Kong',
    'jakarta': 'Indonesia',
    'kuala_lumpur': 'Malaysia',
    'kyoto': 'Japan',
    'manila': 'Philippines',
    'singapore': 'Singapore',
    'tokyo': 'Japan'
}
ig_df.rename(columns={'country': 'city'}, inplace=True)

ig_df['country'] = ig_df['city'].map(city_to_country)

#change to the same month_year format
ig_df['date_only'] = pd.to_datetime(ig_df['date_only'], errors='coerce')
ig_df['month_year'] = ig_df['date_only'].dt.strftime('%Y-%m')

columns_to_keep = [
    "month_year",
    "country",
    "sentiment_score"
]

ig_df = ig_df[columns_to_keep]

#agggregate sentiment score
ig_sentiment = ig_df.groupby(
    ["country", "month_year"]
)["sentiment_score"].mean().reset_index()
ig_sentiment.columns = ["country", "month_year", "ig_sentiment"]

ig_sentiment.head(5)

,country,month_year,ig_sentiment
0,China,2022-10,0.0000
1,China,2023-02,0.0000
2,China,2023-04,0.0000
3,China,2023-05,0.8689
4,China,2023-08,0.0000


In [983]:
# --- Prepare reviews_df ---
# Convert trip_date to datetime and extract month-year
reviews_df["trip_date"] = pd.to_datetime(reviews_df["trip_date"], errors="coerce")
reviews_df["month_year"] = reviews_df["trip_date"].dt.strftime("%Y-%m")
columns_to_keep = [
    "month_year",
    "country",
    "rating"
]
reviews_df = reviews_df[columns_to_keep]
review_agg = reviews_df.groupby(["country", "month_year"])["rating"].mean().reset_index()
review_agg.head(5)

/var/folders/85/3q25ml057m51x2d6fx8s4w740000gn/T/ipykernel_35767/74950663.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  reviews_df["trip_date"] = pd.to_datetime(reviews_df["trip_date"], errors="coerce")


,country,month_year,rating
0,Brunei,2015-09,4.5
1,Brunei,2015-10,5.0
2,Brunei,2016-01,5.0
3,Brunei,2016-02,5.0
4,Brunei,2016-04,4.5


In [984]:
# --- Prepare currency_df ---
currency_df["YearMonth"] = pd.to_datetime(currency_df["YearMonth"], errors="coerce")
currency_df["YearMonth"] = currency_df["YearMonth"].dt.strftime("%Y-%m")
currency_df = currency_df.rename(columns={
    "Currency": "country",
    "YearMonth": "month_year",
    "AverageRate": "avg_currency_rate"
})

unique_codes = currency_df["country"].unique()
print(sorted(unique_codes))
currency_to_country = {
    "BND": "Brunei",
    "IDR": "Indonesia",
    "KHR": "Cambodia",
    "LAK": "Laos",
    "MMK": "Myanmar",
    "MYR": "Malaysia",
    "PHP": "Philippines",
    "SGD": "Singapore",
    "THB": "Thailand",
    "VND": "Vietnam"
}

currency_df["country"] = currency_df["country"].map(currency_to_country)
currency_df.head()

['BND', 'IDR', 'KHR', 'LAK', 'MMK', 'MYR', 'PHP', 'SGD', 'THB', 'VND']


,month_year,country,avg_currency_rate
0,2022-01,Brunei,1.351418
1,2022-01,Indonesia,14327.056094
2,2022-01,Cambodia,4072.629258
3,2022-01,Laos,11272.253421
4,2022-01,Myanmar,1777.280733


In [985]:
# --- Prepare trends_df ---
trend_df = trends_df.rename(columns={"value": "trend_score"})
trend_df.head(5)

,month_year,country,trend_score
0,2021-12,Cambodia,1.0
1,2021-12,Indonesia,17.0
2,2021-12,Malaysia,12.0
3,2021-12,Myanmar,1.0
4,2021-12,Philippines,8.0


In [986]:
# --- Prepare visitors_df ---
visitors_df = visitors_df.rename(columns={"value": "num_visitors"})
visitors_df.head(5)

,country,month_year,num_visitors
0,Thailand,2022-01,134000.0
1,Thailand,2022-02,153000.0
2,Thailand,2022-03,211000.0
3,Thailand,2022-04,293000.0
4,Thailand,2022-05,521000.0


In [987]:
final_df = (
    currency_df
    .merge(trend_df, how="left", on=["country", "month_year"])
    .merge(visitors_df, how="left", on=["country", "month_year"])
    .merge(review_agg, how="left", on=["country", "month_year"])
    .merge(avg_sentiment, how="left", on=["country", "month_year"])
    .merge(ig_sentiment, how="left", on=["country", "month_year"])
)

final_df.head(5)


,month_year,country,avg_currency_rate,trend_score,num_visitors,rating,reddit_sentiment,ig_sentiment
0,2022-01,Brunei,1.351418,NaN,NaN,NaN,NaN,NaN
1,2022-01,Indonesia,14327.056094,12.0,122000.000000,5.0,NaN,0.452788
2,2022-01,Cambodia,4072.629258,2.0,44700.000000,5.0,NaN,NaN
3,2022-01,Laos,11272.253421,NaN,117263.846577,5.0,NaN,NaN
4,2022-01,Myanmar,1777.280733,1.0,11372.000000,5.0,NaN,NaN


In [988]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   month_year         390 non-null    object 
 1   country            390 non-null    object 
 2   avg_currency_rate  390 non-null    float64
 3   trend_score        312 non-null    float64
 4   num_visitors       363 non-null    float64
 5   rating             325 non-null    float64
 6   reddit_sentiment   59 non-null     float64
 7   ig_sentiment       175 non-null    float64
dtypes: float64(6), object(2)
memory usage: 24.5+ KB


In [989]:
final_df[["avg_currency_rate", "trend_score", "num_visitors", "rating","reddit_sentiment","ig_sentiment"]].describe()


,avg_currency_rate,trend_score,num_visitors,rating,reddit_sentiment,ig_sentiment
count,390.000000,312.000000,3.630000e+02,325.000000,59.000000,175.000000
mean,6424.349578,18.250000,7.833493e+05,4.697630,0.567319,0.466014
std,8859.669369,15.600365,7.837684e+05,0.390025,0.366938,0.208626
min,1.296503,1.000000,9.508000e+03,2.000000,0.000000,-0.058188
25%,4.472762,6.750000,1.417285e+05,4.500000,0.187361,0.350400
50%,917.418825,15.000000,4.745032e+05,4.833333,0.709436,0.483633
75%,15054.790128,22.000000,1.230782e+06,4.977273,0.823219,0.600292
max,25533.796176,67.000000,3.710000e+06,5.000000,0.998700,0.970900


In [990]:
final_df.shape

(390, 8)

In [991]:
#Normalisation

scale_cols = ["avg_currency_rate", "trend_score", "num_visitors", "rating", "reddit_sentiment", "ig_sentiment"]

scaler = StandardScaler() 
scaled_features = scaler.fit_transform(final_df[scale_cols].fillna(0))

#outliers
final_df['avg_currency_rate'] = final_df['avg_currency_rate'].clip(upper=final_df['avg_currency_rate'].quantile(0.95))


In [992]:
#log transform

final_df['log_visitors'] = np.log1p(final_df['num_visitors'])
final_df['log_currency'] = np.log1p(final_df['avg_currency_rate'])


In [993]:
#classify sentiment if it helps

final_df['reddit_sentiment_label'] = pd.cut(final_df['reddit_sentiment'], bins=[-0.01, 0.3, 0.7, 1],
                                     labels=['Negative', 'Neutral', 'Positive'])

final_df['ig_sentiment_label'] = pd.cut(final_df['ig_sentiment'], bins=[-0.01, 0.3, 0.7, 1],
                                     labels=['Negative', 'Neutral', 'Positive'])


In [994]:
#trend momentum score

final_df['trend_diff'] = final_df.groupby('country')['trend_score'].diff()

In [995]:
#forward and backward fill for missing sentiment/trend data
#final_df.fillna(method='ffill').fillna(method='bfill')



In [996]:
final_df.head(20)

,month_year,country,avg_currency_rate,trend_score,num_visitors,rating,reddit_sentiment,ig_sentiment,log_visitors,log_currency,reddit_sentiment_label,ig_sentiment_label,trend_diff
0,2022-01,Brunei,1.351418,NaN,NaN,NaN,NaN,NaN,NaN,0.855019,NaN,NaN,NaN
1,2022-01,Indonesia,14327.056094,12.0,122000.000000,5.000000,NaN,0.452788,11.711785,9.569975,NaN,Neutral,NaN
2,2022-01,Cambodia,4072.629258,2.0,44700.000000,5.000000,NaN,NaN,10.707751,8.312290,NaN,NaN,NaN
3,2022-01,Laos,11272.253421,NaN,117263.846577,5.000000,NaN,NaN,11.672190,9.330188,NaN,NaN,NaN
4,2022-01,Myanmar,1777.280733,1.0,11372.000000,5.000000,NaN,NaN,9.338997,7.483402,NaN,NaN,NaN
5,2022-01,Malaysia,4.186455,13.0,30000.000000,4.200000,NaN,NaN,10.308986,1.646051,NaN,NaN,NaN
6,2022-01,Philippines,51.216984,11.0,NaN,4.666667,NaN,NaN,NaN,3.955408,NaN,NaN,NaN
7,2022-01,Singapore,1.351418,31.0,57174.000000,NaN,NaN,0.657180,10.953872,0.855019,NaN,Neutral,NaN
8,2022-01,Thailand,33.276252,19.0,134000.000000,5.000000,NaN,0.527889,11.805603,3.534453,NaN,Neutral,NaN
9,2022-01,Vietnam,22748.329719,7.0,19727.000000,NaN,NaN,0.523843,9.889794,10.032291,NaN,Neutral,NaN
